In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation 
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



## Import Dataset

In [2]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 224
batch_size = 64
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

## Import Model

In [3]:
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs 
model = tf.keras.applications.vgg19.VGG19(weights='imagenet',
                                             input_tensor=x,include_top=False)

model.trainable = False
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2, name="top_dropout")(x)
outputs = layers.Dense(10, activation="softmax", name="pred")(x)
model = Model(inputs = inputs, outputs = outputs)
model.summary()

80142336/80134624 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

## Transfer Learning on Model

In [4]:
model_name = 'vgg16_cifar10.h5'
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(1e-2),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(ds_train,
              epochs=30,
              validation_data=ds_test,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/30
781/781 [==============================] - 67s 79ms/step - loss: 0.8352 - accuracy: 0.7260 - val_loss: 0.6331 - val_accuracy: 0.7812
Epoch 2/30
781/781 [==============================] - 62s 79ms/step - loss: 0.7153 - accuracy: 0.7539 - val_loss: 0.6175 - val_accuracy: 0.7926
Epoch 3/30
781/781 [==============================] - 62s 79ms/step - loss: 0.7063 - accuracy: 0.7589 - val_loss: 0.6191 - val_accuracy: 0.7904
Epoch 4/30
781/781 [==============================] - 62s 80ms/step - loss: 0.7097 - accuracy: 0.7565 - val_loss: 0.6395 - val_accuracy: 0.7830
Epoch 5/30
781/781 [==============================] - 62s 79ms/step - loss: 0.7077 - accuracy: 0.7593 - val_loss: 0.6209 - val_accuracy: 0.7877
Epoch 6/30
781/781 [==============================] - 62s 79ms/step - loss: 0.7101 - accuracy: 0.7539 - val_loss: 0.6274 - val_accuracy: 0.7863
Epoch 7/30
781/781 [==============================] - 62s 79ms/step - loss: 0.7088 - accuracy: 0.7564 - val_loss: 0.6328 - val_accuracy:

In [5]:
def unfreeze_model(model):
    for layer in model.layers[-15:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(ds_train,
              epochs=30,
              validation_data=ds_test,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/30
781/781 [==============================] - 96s 122ms/step - loss: 0.5114 - accuracy: 0.8294 - val_loss: 0.5572 - val_accuracy: 0.8236
Epoch 2/30
781/781 [==============================] - 95s 122ms/step - loss: 0.2658 - accuracy: 0.9095 - val_loss: 0.5730 - val_accuracy: 0.8257
Epoch 3/30
781/781 [==============================] - 95s 122ms/step - loss: 0.1697 - accuracy: 0.9419 - val_loss: 0.5171 - val_accuracy: 0.8429
Epoch 4/30
781/781 [==============================] - 95s 122ms/step - loss: 0.1182 - accuracy: 0.9602 - val_loss: 0.4575 - val_accuracy: 0.8684
Epoch 5/30
781/781 [==============================] - 95s 122ms/step - loss: 0.0886 - accuracy: 0.9696 - val_loss: 0.3965 - val_accuracy: 0.8875
Epoch 6/30
781/781 [==============================] - 95s 122ms/step - loss: 0.0787 - accuracy: 0.9729 - val_loss: 0.4020 - val_accuracy: 0.8930
Epoch 7/30
781/781 [==============================] - 95s 122ms/step - loss: 0.0539 - accuracy: 0.9820 - val_loss: 0.3729 - val_ac

In [6]:
new_model = load_model(model_name)

In [7]:
new_model.evaluate(ds_test)

156/156 [==============================] - 10s 64ms/step - loss: 0.3288 - accuracy: 0.9233


[0.32876715064048767, 0.9232772588729858]